# Question 1

In [8]:
! pip install requests
! pip install pandas
! pip install psycopg[binary]
! pip install psycopg2


[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


   ---------------------------------------- 0.0/2.8 MB ? eta -:--:--
   ----------- ---------------------------- 0.8/2.8 MB 8.0 MB/s eta 0:00:01
   ---------------------------------------- 2.8/2.8 MB 12.4 MB/s  0:00:00



[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import requests

url = "https://cricbuzz-cricket.p.rapidapi.com/teams/v1/2/players"

headers = {
	"x-rapidapi-key": "5d00263db8msh6b461e04fff0e6bp1aa84cjsn57ff489cc9c5",
	"x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
}

response = requests.get(url, headers=headers)

print(response.json())


{'player': [{'name': 'BATSMEN', 'imageId': 174146}, {'id': '11808', 'name': 'Shubman Gill', 'imageId': 616515, 'battingStyle': 'Right-hand bat', 'bowlingStyle': 'Right-arm offbreak'}, {'id': '13940', 'name': 'Yashasvi Jaiswal', 'imageId': 591942, 'battingStyle': 'Left-hand bat', 'bowlingStyle': 'Right-arm legbreak'}, {'id': '13866', 'name': 'Sai Sudharsan', 'imageId': 717782, 'battingStyle': 'Left-hand bat', 'bowlingStyle': 'Right-arm legbreak'}, {'id': '576', 'name': 'Rohit Sharma', 'imageId': 616514, 'battingStyle': 'Right-hand bat', 'bowlingStyle': 'Right-arm offbreak'}, {'id': '1413', 'name': 'Virat Kohli', 'imageId': 616517, 'battingStyle': 'Right-hand bat', 'bowlingStyle': 'Right-arm medium'}, {'id': '7915', 'name': 'Suryakumar Yadav', 'imageId': 846028, 'battingStyle': 'Right-hand bat', 'bowlingStyle': 'Right-arm offbreak'}, {'id': '9428', 'name': 'Shreyas Iyer', 'imageId': 616518, 'battingStyle': 'Right-hand bat', 'bowlingStyle': 'Right-arm legbreak'}, {'id': '10896', 'name': '

In [3]:
players_data = response.json()
print(players_data)

{'player': [{'name': 'BATSMEN', 'imageId': 174146}, {'id': '11808', 'name': 'Shubman Gill', 'imageId': 616515, 'battingStyle': 'Right-hand bat', 'bowlingStyle': 'Right-arm offbreak'}, {'id': '13940', 'name': 'Yashasvi Jaiswal', 'imageId': 591942, 'battingStyle': 'Left-hand bat', 'bowlingStyle': 'Right-arm legbreak'}, {'id': '13866', 'name': 'Sai Sudharsan', 'imageId': 717782, 'battingStyle': 'Left-hand bat', 'bowlingStyle': 'Right-arm legbreak'}, {'id': '576', 'name': 'Rohit Sharma', 'imageId': 616514, 'battingStyle': 'Right-hand bat', 'bowlingStyle': 'Right-arm offbreak'}, {'id': '1413', 'name': 'Virat Kohli', 'imageId': 616517, 'battingStyle': 'Right-hand bat', 'bowlingStyle': 'Right-arm medium'}, {'id': '7915', 'name': 'Suryakumar Yadav', 'imageId': 846028, 'battingStyle': 'Right-hand bat', 'bowlingStyle': 'Right-arm offbreak'}, {'id': '9428', 'name': 'Shreyas Iyer', 'imageId': 616518, 'battingStyle': 'Right-hand bat', 'bowlingStyle': 'Right-arm legbreak'}, {'id': '10896', 'name': '

In [4]:
players_data
players_data['player']

[{'name': 'BATSMEN', 'imageId': 174146},
 {'id': '11808',
  'name': 'Shubman Gill',
  'imageId': 616515,
  'battingStyle': 'Right-hand bat',
  'bowlingStyle': 'Right-arm offbreak'},
 {'id': '13940',
  'name': 'Yashasvi Jaiswal',
  'imageId': 591942,
  'battingStyle': 'Left-hand bat',
  'bowlingStyle': 'Right-arm legbreak'},
 {'id': '13866',
  'name': 'Sai Sudharsan',
  'imageId': 717782,
  'battingStyle': 'Left-hand bat',
  'bowlingStyle': 'Right-arm legbreak'},
 {'id': '576',
  'name': 'Rohit Sharma',
  'imageId': 616514,
  'battingStyle': 'Right-hand bat',
  'bowlingStyle': 'Right-arm offbreak'},
 {'id': '1413',
  'name': 'Virat Kohli',
  'imageId': 616517,
  'battingStyle': 'Right-hand bat',
  'bowlingStyle': 'Right-arm medium'},
 {'id': '7915',
  'name': 'Suryakumar Yadav',
  'imageId': 846028,
  'battingStyle': 'Right-hand bat',
  'bowlingStyle': 'Right-arm offbreak'},
 {'id': '9428',
  'name': 'Shreyas Iyer',
  'imageId': 616518,
  'battingStyle': 'Right-hand bat',
  'bowlingStyl

In [5]:
indian_players = []
current_role = None
for player in players_data['player']:    
    if 'id' not in player:
        current_role = player['name']
        continue
    role = player.get('role') or player.get('playingRole') or current_role
    indian_players.append({
        'id': player['id'],
        'name': player['name'],
        'role': role,
        'batting_style': player.get('battingStyle'),
        'bowling_style': player.get('bowlingStyle')
    })

In [2]:
import pandas as pd
indian_players_df = pd.DataFrame(indian_players)

NameError: name 'indian_players' is not defined

In [2]:
import psycopg2
import requests
DB_CONFIG = {
    'host': 'localhost',
    'database': 'guviproj',
    'user': 'postgres',
    'password': 'haha',
    'port': 5432
    }   

In [15]:
conn = psycopg2.connect(**DB_CONFIG)
cur = conn.cursor()
print("Connected to the database successfully")

Connected to the database successfully


In [16]:
cur.execute("""
    CREATE TABLE IF NOT EXISTS indians_players_list (
    id SERIAL PRIMARY KEY,
    name VARCHAR(100),
    role VARCHAR(100),
    batting_style VARCHAR(100),
    bowling_style VARCHAR(100)
    )
""")
conn.commit()
print("Table created successfully")    

Table created successfully


In [17]:
count = 0
for player in indian_players:
    try:
        id = int(player['id'])
        name = player['name']
        role = player['role']
        batting_style = player['batting_style']
        bowling_style = player['bowling_style']
        cur.execute("""
            INSERT INTO indians_players_list (id, name, role, batting_style, bowling_style)
            VALUES (%s, %s, %s, %s, %s)
            ON CONFLICT (id) DO NOTHING        
        """, (
            id,
            name,
            role,
            batting_style,
            bowling_style
        ))
        count += 1
    except Exception as e:
        print(f"Error inserting player {player}: {e}")
conn.commit()
print(f"{count} records inserted successfully")
cur.close()
conn.close()

37 records inserted successfully


# Question 2

In [18]:
import requests

url = "https://cricbuzz-cricket.p.rapidapi.com/matches/v1/recent"

headers = {
	"x-rapidapi-key": "5d00263db8msh6b461e04fff0e6bp1aa84cjsn57ff489cc9c5",
	"x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
}

response = requests.get(url, headers=headers)

print(response.json())
recent_matches = response.json()
print(recent_matches)

{'typeMatches': [{'matchType': 'International', 'seriesMatches': [{'seriesAdWrapper': {'seriesId': 11253, 'seriesName': "ICC Men's T20 World Cup 2026", 'matches': [{'matchInfo': {'matchId': 139227, 'seriesId': 11253, 'seriesName': "ICC Men's T20 World Cup 2026", 'matchDesc': '28th Match, Group D', 'matchFormat': 'T20', 'startDate': '1771219800000', 'endDate': '1771232400000', 'state': 'Complete', 'status': 'Afghanistan won by 5 wkts', 'team1': {'teamId': 7, 'teamName': 'United Arab Emirates', 'teamSName': 'UAE', 'imageId': 776242}, 'team2': {'teamId': 96, 'teamName': 'Afghanistan', 'teamSName': 'AFG', 'imageId': 776177}, 'venueInfo': {'id': 51, 'ground': 'Arun Jaitley Stadium', 'city': 'Delhi', 'timezone': '+05:30', 'latitude': '28.637891', 'longitude': '77.243073'}, 'currBatTeamId': 96, 'seriesStartDt': '1770422400000', 'seriesEndDt': '1773100800000', 'isTimeAnnounced': True, 'stateTitle': 'Complete'}, 'matchScore': {'team1Score': {'inngs1': {'inningsId': 1, 'runs': 160, 'wickets': 9,

In [19]:
type_matches = recent_matches['typeMatches']
series_matches = type_matches[0]['seriesMatches'][0]
series_wrapper = series_matches['seriesAdWrapper']
match_list = series_wrapper['matches']
first_match = match_list[0]
print(first_match['matchInfo'])

{'matchId': 139227, 'seriesId': 11253, 'seriesName': "ICC Men's T20 World Cup 2026", 'matchDesc': '28th Match, Group D', 'matchFormat': 'T20', 'startDate': '1771219800000', 'endDate': '1771232400000', 'state': 'Complete', 'status': 'Afghanistan won by 5 wkts', 'team1': {'teamId': 7, 'teamName': 'United Arab Emirates', 'teamSName': 'UAE', 'imageId': 776242}, 'team2': {'teamId': 96, 'teamName': 'Afghanistan', 'teamSName': 'AFG', 'imageId': 776177}, 'venueInfo': {'id': 51, 'ground': 'Arun Jaitley Stadium', 'city': 'Delhi', 'timezone': '+05:30', 'latitude': '28.637891', 'longitude': '77.243073'}, 'currBatTeamId': 96, 'seriesStartDt': '1770422400000', 'seriesEndDt': '1773100800000', 'isTimeAnnounced': True, 'stateTitle': 'Complete'}


In [20]:
matches = []

for type_match in recent_matches.get('typeMatches', []):

    for series_match in type_match.get('seriesMatches', []):
        series_wrapper = series_match.get('seriesAdWrapper', {})
        series_id = series_wrapper.get('seriesId')
        series_name = series_wrapper.get('seriesName')

        for match in series_wrapper.get('matches', []):
            info = match.get('matchInfo', {})
            match_id = info.get('matchId')
            match_desc = info.get('matchDesc')
            match_format = info.get('matchFormat')
            start_date = info.get('startDate')
            status = info.get('status')
            team1 = info.get('team1', {}).get('teamName')
            team2 = info.get('team2', {}).get('teamName')
            venue = info.get('venueInfo', {}).get('ground')

            matches.append({
                'series_id': series_id,
                'series_name': series_name,
                'match_id': match_id,
                'match_desc': match_desc,
                'match_format': match_format,
                'start_date': start_date,
                'status': status,
                'team1': team1,
                'team2': team2,
                'venue': venue
            })

In [21]:
conn = psycopg2.connect(**DB_CONFIG)
cur = conn.cursor()
print("Connected to the database successfully")

Connected to the database successfully


In [22]:
cur.execute("""
    CREATE TABLE IF NOT EXISTS recents_matches (
    series_id INTEGER,
    series_name VARCHAR(255),
    match_id INTEGER,
    match_desc TEXT,
    match_format VARCHAR(100),
    start_date DATE,
    status VARCHAR(100),
    team1 VARCHAR(255),
    team2 VARCHAR(255),
    venue VARCHAR(255)
    )
""")
conn.commit()
print("Table created successfully")  

Table created successfully


In [23]:
from datetime import datetime

count = 0
errors = []

for data in matches:
    try:
        series_id = data['series_id']
        series_name = data['series_name']
        match_id = data['match_id']
        match_desc = data['match_desc']
        match_format = data['match_format']
        start_date = data['start_date']
        status = data['status']
        team1 = data['team1']
        team2 = data['team2']
        venue = data['venue']
        
        
        if start_date:
            
            start_date = datetime.fromtimestamp(int(start_date) / 1000).date()
        else:
            start_date = None
        
        cur.execute("""
            INSERT INTO recents_matches (series_id, series_name, match_id, match_desc, match_format, start_date, status, team1, team2, venue)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """, (series_id, series_name, match_id, match_desc, match_format, start_date, status, team1, team2, venue))
        count += 1
    except Exception as e:
        errors.append(str(e))

conn.commit()
print(f"{count} records inserted successfully")
if errors:
    print(f"\nUnique errors encountered:")
    for error in set(errors[:5]): 
        print(f"  - {error}")
        
cur.close()
conn.close()

59 records inserted successfully


# Question 3

In [24]:
url = "https://cricbuzz-cricket.p.rapidapi.com/stats/v1/topstats/0"

querystring = {"statsType":"mostRuns","matchType":"2"}

headers = {
	"x-rapidapi-key": "5d00263db8msh6b461e04fff0e6bp1aa84cjsn57ff489cc9c5",
	"x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

print(response.json())
top_odi_runs = response.json()
top_odi = top_odi_runs['values']
player_ids = [int(item['values'][0]) for item in top_odi]

{'filter': {'matchtype': [{'matchTypeId': '1', 'matchTypeDesc': 'test'}, {'matchTypeId': '2', 'matchTypeDesc': 'odi'}, {'matchTypeId': '3', 'matchTypeDesc': 't20i'}], 'team': [{'id': '2', 'teamShortName': 'IND'}, {'id': '27', 'teamShortName': 'IRE'}, {'id': '3', 'teamShortName': 'PAK'}, {'id': '4', 'teamShortName': 'AUS'}, {'id': '5', 'teamShortName': 'SL'}, {'id': '6', 'teamShortName': 'BAN'}, {'id': '9', 'teamShortName': 'ENG'}, {'id': '10', 'teamShortName': 'WI'}, {'id': '11', 'teamShortName': 'RSA'}, {'id': '12', 'teamShortName': 'ZIM'}, {'id': '13', 'teamShortName': 'NZ'}, {'id': '96', 'teamShortName': 'AFG'}], 'selectedMatchType': 'odi'}, 'headers': ['Batter', 'M', 'I', 'R', 'Avg'], 'values': [{'values': ['25', 'Tendulkar', '463', '452', '18426', '44.83']}, {'values': ['1413', 'Virat Kohli', '311', '299', '14797', '58.72']}, {'values': ['104', 'Sangakkara', '404', '380', '14234', '41.99']}, {'values': ['38', 'R Ponting', '375', '365', '13704', '42.04']}, {'values': ['102', 'S Jay

In [25]:
player_data = {}

for vid in player_ids:
    try:
        url = f"https://cricbuzz-cricket.p.rapidapi.com/stats/v1/player/{vid}/batting"
    
        response = requests.get(url, headers=headers, timeout=5)

        print(f"\n===== Series ID {vid} =====")

        if response.status_code == 200:
            player_data[vid] = response.json()
            print(player_data[vid])
        else:
            print("Failed:", response.status_code)

    except Exception as e:
        print("Error for", vid, e)

odi_hundreds = {}

for pid, pdata in player_data.items():
    for row in pdata['values']:
        if row['values'][0] == '100s':
            odi_hundreds[pid] = int(row['values'][2])
            break
odi_hundreds        


===== Series ID 25 =====
{'headers': ['ROWHEADER', 'Test', 'ODI', 'T20', 'IPL'], 'values': [{'values': ['Matches', '200', '463', '1', '78']}, {'values': ['Innings', '329', '452', '1', '78']}, {'values': ['Runs', '15921', '18426', '10', '2334']}, {'values': ['Balls', '29437', '21367', '12', '1948']}, {'values': ['Highest', '248', '200', '10', '100']}, {'values': ['Average', '53.79', '44.83', '10', '33.83']}, {'values': ['SR', '54.09', '86.24', '83.34', '119.82']}, {'values': ['Not Out', '33', '41', '0', '9']}, {'values': ['Fours', '2058', '2016', '2', '295']}, {'values': ['Sixes', '69', '195', '0', '29']}, {'values': ['Ducks', '14', '20', '0', '4']}, {'values': ['50s', '68', '96', '0', '13']}, {'values': ['100s', '51', '49', '0', '1']}, {'values': ['200s', '6', '1', '0', '0']}, {'values': ['300s', '0', '0', '0', '0']}, {'values': ['400s', '0', '0', '0', '0']}], 'appIndex': {'seoTitle': 'Sachin Tendulkar Profile - Cricbuzz | Cricbuzz.com', 'webURL': 'http://www.cricbuzz.com/profiles/25/

{25: 49,
 1413: 54,
 104: 25,
 38: 30,
 102: 28,
 101: 19,
 35: 10,
 213: 17,
 576: 33,
 29: 22,
 27: 12,
 265: 10,
 247: 25,
 240: 19,
 105: 22,
 34: 15,
 36: 16,
 370: 25,
 3864: 7,
 3531: 11}

In [26]:
rows = []

for pid, pdata in player_data.items():
    name = pdata['appIndex']['seoTitle'].split(' Profile')[0]

    odi_runs = odi_avg = odi_100s = None

    for row in pdata['values']:
        label = row['values'][0]
        if label == 'Runs':
            odi_runs = int(row['values'][2])
        elif label == 'Average':
            odi_avg = float(row['values'][2])
        elif label == '100s':
            odi_100s = int(row['values'][2])

    rows.append({
        'Player Name': name,
        'Runs': odi_runs,
        'Average': odi_avg,
        'Centuries': odi_100s
    })

top_odi_10_df = pd.DataFrame(rows).sort_values('Runs', ascending=False).head(10)
top_odi_10_df

,Player Name,Runs,Average,Centuries
0,Sachin Tendulkar,18426,44.83,49
1,Virat Kohli,14797,58.72,54
2,Kumar Sangakkara,14234,41.99,25
3,Ricky Ponting,13704,42.04,30
4,Sanath Jayasuriya,13430,32.13,28
5,Mahela Jayawardene,12650,33.38,19
6,Inzamam-ul-Haq,11739,39.53,10
7,Jacques Kallis,11579,44.36,17
8,Rohit Sharma,11577,48.85,33
9,Sourav Ganguly,11363,40.73,22


In [27]:
conn = psycopg2.connect(**DB_CONFIG)
cur = conn.cursor()
print("Connected to the database successfully")

Connected to the database successfully


In [28]:
# Create table
cur.execute("""
CREATE TABLE IF NOT EXISTS "Top 10 odi scorers" (
    player_name VARCHAR(100),
    runs INTEGER,
    average NUMERIC(5,2),
    centuries INTEGER
)
""")

# Insert query
insert_query = """
INSERT INTO "Top 10 odi scorers"
(player_name, runs, average, centuries)
VALUES (%s, %s, %s, %s)
"""

# Convert DataFrame → tuples
data = list(top_odi_10_df.itertuples(index=False, name=None))

# Insert
cur.executemany(insert_query, data)

conn.commit()
cur.close()


# Question 4

In [43]:

headers = {
	"x-rapidapi-key": "71e13280e1mshf1b57ab53de3596p1f62c8jsndbbb9f796741",
	"x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
}
venue_data = {}
conn = psycopg2.connect(**DB_CONFIG)
cur = conn.cursor()
print("Connected to the database successfully")

# Create table
cur.execute("""
CREATE TABLE IF NOT EXISTS "Venues with 20k Cap" (
    venue_name VARCHAR(100),
    capacity VARCHAR(100),
    city VARCHAR(100),
    country VARCHAR(100)
)
""")
for vid in range(1, 50):
    try:
        url = f"https://cricbuzz-cricket.p.rapidapi.com/venues/v1/{vid}"
        response = requests.get(url, headers=headers, timeout=5)

        print(f"\n===== Venue ID {vid} =====")

        if response.status_code == 200:
            venue_data[vid] = response.json()
            print(venue_data[vid])
            name = venue_data[vid]['ground']
            capacity = venue_data[vid]['capacity']
            city = venue_data[vid]['city']
            country = venue_data[vid]['country']
            if capacity is None or capacity == 'N/A' or int(capacity) < 25000:
                print(f"Skipping {name} due to capacity {capacity}")
                continue
            else: 
                insert_query = """
                        INSERT INTO "Venues with 20k Cap"
                        (venue_name, capacity, city, country)
                        VALUES (%s, %s, %s, %s)
                        """
                cur.execute(insert_query, (name, capacity, city, country))
        else:
            print("Failed:", response.status_code)

    except Exception as e:
        print("Error for", vid, e)

conn.commit()
cur.close()
conn.close()

Connected to the database successfully

===== Venue ID 1 =====
Failed: 204

===== Venue ID 2 =====
Failed: 204

===== Venue ID 3 =====
Failed: 204

===== Venue ID 4 =====
Failed: 204

===== Venue ID 5 =====
Failed: 204

===== Venue ID 6 =====
Failed: 204

===== Venue ID 7 =====
Failed: 204

===== Venue ID 8 =====
Failed: 204

===== Venue ID 9 =====
Failed: 204

===== Venue ID 10 =====
{'ground': 'Brabourne Stadium', 'city': 'Mumbai', 'country': 'India', 'timezone': '+05:30', 'established': 1937, 'capacity': '20,000', 'ends': 'Pavilion End, Church Gate End', 'homeTeam': 'Mumbai', 'floodlights': True, 'profile': "<b>Venue Description</b>:\n\nOwned by the Cricket Club of India (CCI), Brabourne Stadium is India's first permanent sporting venue. Established in 1937, the stadium was built on reclaimed land along the Marine Drive in South Mumbai. It is named after Lord Brabourne, then the Governor of Bombay. The story goes that the land was presented to the CCI after Lord Brabourne was wooed 

# Question 5

In [8]:
headers = {
	"x-rapidapi-key": "6659fc426bmshceca4f0c502e967p189b21jsn697115d9f031",
	"x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
}
venue_data = {}
conn = psycopg2.connect(**DB_CONFIG)
cur = conn.cursor()
print("Connected to the database successfully")

# Create table
cur.execute("""
CREATE TABLE IF NOT EXISTS "teams with wins" (
    team_name VARCHAR(100),
    num_of_wins VARCHAR(100)
)
""")

url = f"https://cricbuzz-cricket.p.rapidapi.com/teams/v1/international"
response = requests.get(url, headers=headers, timeout=5)
print(response.json())

for team in response.json()['list']:
    num_of_wins = 0
    team_name = team['teamName']
    team_id = team.get('teamId', None)
    if team_id is not None:
        get_matchInfo_url = f"https://cricbuzz-cricket.p.rapidapi.com/teams/v1/{team_id}/results"
        match_response = requests.get(get_matchInfo_url, headers=headers, timeout=5)
        for matches in match_response.json().get('teamMatchesData', []):
            matchDetailsMap = matches.get('matchDetailsMap', {})
            print("inside matchDetailsMap")
            for match in matchDetailsMap.get('match', []):
                match_info = match.get('matchInfo', {})
                status = match_info.get('status', '')
                print(f"Team: {team_name}, Status: {status}")
                if team_name in status:
                    num_of_wins += 1


        cur.execute("""
            INSERT INTO "teams with wins" (team_name, num_of_wins)
            VALUES (%s, %s)
            """, (team_name, num_of_wins))


conn.commit()
cur.close()
conn.close()

Connected to the database successfully
{'list': [{'teamName': 'Test Teams'}, {'teamId': 2, 'teamName': 'India', 'teamSName': 'IND', 'imageId': 776162, 'countryName': 'India'}, {'teamId': 96, 'teamName': 'Afghanistan', 'teamSName': 'AFG', 'imageId': 776177}, {'teamId': 27, 'teamName': 'Ireland', 'teamSName': 'IRE', 'imageId': 839366}, {'teamId': 3, 'teamName': 'Pakistan', 'teamSName': 'PAK', 'imageId': 776308, 'countryName': 'Pakistan'}, {'teamId': 4, 'teamName': 'Australia', 'teamSName': 'AUS', 'imageId': 776202, 'countryName': 'Australia'}, {'teamId': 5, 'teamName': 'Sri Lanka', 'teamSName': 'SL', 'imageId': 776254, 'countryName': 'Sri Lanka'}, {'teamId': 6, 'teamName': 'Bangladesh', 'teamSName': 'BAN', 'imageId': 776210}, {'teamId': 9, 'teamName': 'England', 'teamSName': 'ENG', 'imageId': 776237}, {'teamId': 10, 'teamName': 'West Indies', 'teamSName': 'WI', 'imageId': 776191}, {'teamId': 11, 'teamName': 'South Africa', 'teamSName': 'RSA', 'imageId': 776287}, {'teamId': 12, 'teamName'

# Question 6

In [28]:
headers = {
	"x-rapidapi-key": "3ddef92f6emsh8301b1a8e1fd478p15bb8bjsnd0bb5446cadc",
	"x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
}
venue_data = {}
conn = psycopg2.connect(**DB_CONFIG)
cur = conn.cursor()
print("Connected to the database successfully")

# Create table
cur.execute("""
CREATE TABLE IF NOT EXISTS "category count" (
    batsman VARCHAR(100),
    bowler VARCHAR(100),
    all_rounder VARCHAR(100),
    wicket_keeper VARCHAR(100)
)
""")

url = f"https://cricbuzz-cricket.p.rapidapi.com/teams/v1/international"
response = requests.get(url, headers=headers, timeout=5)
role_count = {"BATSMEN":0,"BOWLER":0,"ALL ROUNDER":0,"WICKET KEEPER":0}
current_role = None

for team in response.json()['list']:
    team_id = team.get('teamId',None)
    if team_id is not None:
        print(team_id)
        url = f"https://cricbuzz-cricket.p.rapidapi.com/teams/v1/{team_id}/players"
        response = requests.get(url, headers=headers)
        if response.status_code != 200:
            continue
        player_data = response.json()
        players_data = player_data['player']
        
        for item in players_data:
    
            if 'id' not in item and 'name' in item:
                current_role = item['name']
            elif 'id' in item and current_role is not None:
                role_count[current_role] += 1

cur.execute("""
INSERT INTO "category count" (batsman, bowler, all_rounder, wicket_keeper)
VALUES (%s, %s, %s, %s)
""", (
    role_count["BATSMEN"],
    role_count["BOWLER"],
    role_count["ALL ROUNDER"],
    role_count["WICKET KEEPER"]
))

conn.commit()
cur.close()
conn.close()





Connected to the database successfully
2
96
27
3
4
5
6
9
10
11
12
13
71
72
77
161
185
190
287
298
300
303
304
343
527
529
541
44
26
7
8
14
15
23
24
25
675


# Question 7

In [37]:
url = "https://cricbuzz-cricket.p.rapidapi.com/stats/v1/topstats/0"

conn = psycopg2.connect(**DB_CONFIG)
cur = conn.cursor()
print("Connected to the database successfully")

# Create table
cur.execute("""
CREATE TABLE IF NOT EXISTS "most runs" (
    odi VARCHAR(100),
    t20  VARCHAR(100),
    test VARCHAR(100)
)
""")

querystring_TEST = {"statsType":"mostRuns","matchType":"1"}
querystring_ODI = {"statsType":"mostRuns","matchType":"2"}
querystring_T20 = {"statsType":"mostRuns","matchType":"3"}

headers = {
	"x-rapidapi-key": "70a8792460msh629f8e0af8cc36bp17accbjsn7c270b8814ee",
	"x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
}

response_TEST = requests.get(url, headers=headers, params=querystring_TEST)
response_ODI = requests.get(url, headers=headers, params=querystring_ODI)
response_T20 = requests.get(url, headers=headers, params=querystring_T20)

print(response_ODI)

odi_response = response_ODI.json()
odi_values = odi_response['values']

t20_response = response_T20.json()
t20_values = t20_response['values']

test_response = response_TEST.json()
test_values = test_response['values']

formats = {
    "Test": test_values,
    "ODI": odi_values,
    "T20I": t20_values
}

result = {"Test": 0,
    "ODI": 0,
    "T20I": 0}

for format_name, data in formats.items():
    max_runs = 0
    top_player = None

    for row in data:
        runs = int(row['values'][4])
        player = row['values'][1]

        if runs > max_runs:
            max_runs = runs
            top_player = player

    result[format_name] = max_runs

print(result)

cur.execute("""
INSERT INTO "most runs" (odi, t20, test)
VALUES (%s, %s, %s)
""", (
    result["ODI"],
    result["T20I"],
    result["Test"],
    
))
conn.commit()
cur.close()
conn.close()

    



Connected to the database successfully
<Response [200]>
{'Test': 15921, 'ODI': 18426, 'T20I': 4716}


# Question 8

In [ ]:
from datetime import datetime

url = "https://cricbuzz-cricket.p.rapidapi.com/series/v1/archives/international"

headers = {
	"x-rapidapi-key": "70a8792460msh629f8e0af8cc36bp17accbjsn7c270b8814ee",
	"x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
}


conn = psycopg2.connect(**DB_CONFIG)
cur = conn.cursor()
print("Connected to the database successfully")

cur.execute("""
CREATE TABLE IF NOT EXISTS "2024 series data" (
    id INT,
    series_name  VARCHAR(100),
    host_country VARCHAR(100),
    match_type VARCHAR(100),
    start_date VARCHAR(100),
    total_matches VARCHAR(100)
)
""")

years = ["2024"]

results = {}   

for year in years:
    querystring = {"year": year}
    response = requests.get(url, headers=headers, params=querystring)
    print(response)
    A_list = response.json()
    series_list = A_list['seriesMapProto'][0]['series']
    ids = [item['id'] for item in series_list]
    first_30 = ids[:30]
    series_data = {}

    for vid in first_30:
        try:
            url = f"https://cricbuzz-cricket.p.rapidapi.com/series/v1/{vid}"
    
            response = requests.get(url, headers=headers, timeout=5)

            if response.status_code == 200:
                series_data[vid] = response.json()
            else:
                print("Failed:", response.status_code)

        except Exception as e:
            print("Error for", vid, e)
    from datetime import datetime


for series_id in first_30:
    if series_id not in series_data:
        continue

    match_details = series_data[series_id].get("matchDetails", [])

    series_name = None
    start_date = None
    total_matches = 0
    match_formats = set()
    host_country = None

    for item in match_details:
        if "matchDetailsMap" not in item:
            continue

        for match in item["matchDetailsMap"]["match"]:
            info = match["matchInfo"]          
            series_name = info["seriesName"]           
            match_date = datetime.fromtimestamp(
                int(info["startDate"]) / 1000
            )

            if not start_date or match_date < start_date:
                start_date = match_date

        
            match_formats.add(info["matchFormat"])

           
            total_matches += 1

            
            if not host_country:
                host_country = info["venueInfo"]["city"]

    results[series_id] = {
        "series_name": series_name,
        "host_country": host_country,
        "match_type": ", ".join(sorted(match_formats)),
        "start_date": start_date.date() if start_date else None,
        "total_matches": total_matches
    }
for series_id, data in results.items():
    cur.execute("""
        INSERT INTO "2024 series data" (id, series_name, host_country, match_type, start_date, total_matches)
        VALUES (%s, %s, %s, %s, %s, %s)
    """, (
        series_id,
        data["series_name"],
        data["host_country"],
        data["match_type"],
        data["start_date"],
        data["total_matches"]
    ))
conn.commit()
cur.close()
conn.close()
print("Data inserted successfully")

    



Connected to the database successfully
<Response [200]>
Data inserted successfully


# Intermediate

# Question 9

In [ ]:
teamIds = [ 2, 4, 9]

headers = {
	"x-rapidapi-key": "8d48575b7bmshd9f62d0dd3b4216p1c6b91jsnf28606b10f93",
	"x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
}

conn = psycopg2.connect(**DB_CONFIG)
cur = conn.cursor()
print("Connected to the database successfully")

cur.execute("""
CREATE TABLE IF NOT EXISTS "players with 1k runs and 50 wickets" (
    player_name  VARCHAR(100),
    format VARCHAR(100),
    total_runs VARCHAR(100),
    total_wickets VARCHAR(100)
)
""")

rounder_ids = {}

for team_id in teamIds:
    url = f"https://cricbuzz-cricket.p.rapidapi.com/teams/v1/{team_id}/players"
    response = requests.get(url, headers=headers)
    if(response.status_code == 204):
        continue
    response_json = response.json()
    players = response_json['player']

    collect = False

    for p in players:
        if p.get('name') == 'ALL ROUNDER':
            collect = True
            continue
        elif p.get('name') == 'WICKET KEEPER':
            break

        if collect and 'id' in p:
            rounder_ids[int(p['id'])] = p.get('name')                    

print(rounder_ids)

def extract_runs(player_data):
    runs = {}

    for row in player_data['values']:
        metric = row['values'][0]
        if metric == 'Runs':
            runs = {
                'Test': int(row['values'][1]),
                'ODI': int(row['values'][2]),
                'T20': int(row['values'][3]),
                'IPL': int(row['values'][4])
            }
    return runs

def extract_wickets(player_data):
    wickets = {}

    for row in player_data['values']:
        metric = row['values'][0]
        if metric == 'Wickets':
            wickets = {
                'Test': int(row['values'][1]),
                'ODI': int(row['values'][2]),
                'T20': int(row['values'][3]),
                'IPL': int(row['values'][4])
            }
    return wickets

bowling_data = {}
batting_data = {}

for player_id,player_name in rounder_ids.items():
    try:
        bowling_url = f"https://cricbuzz-cricket.p.rapidapi.com/stats/v1/player/{player_id}/bowling"

        response_bowling = requests.get(bowling_url, headers=headers, timeout=5)

        if response_bowling.status_code == 200:
            bowling_data[player_id] = response_bowling.json()
        else:
            print("Failed:", response_bowling.status_code)

    except Exception as e:
        print("Error for", player_id, e)
    try:
        batting_url = f"https://cricbuzz-cricket.p.rapidapi.com/stats/v1/player/{player_id}/batting"

        response_batting = requests.get(batting_url, headers=headers, timeout=5)

        if response_batting.status_code == 200:
            batting_data[player_id] = response_batting.json()
        else:
            print("Failed:", response_batting.status_code)

    except Exception as e:
        print("Error for", player_id, e)

    formats = ["Test","ODI","T20","IPL"]
    for player_id, player_data in batting_data.items():
        total_runs = extract_runs(player_data)
    for player_id, player_data in bowling_data.items():
        total_wickets = extract_wickets(player_data)

    for format in formats:
        if total_runs[format] > 1000 and total_wickets[format] > 50:
            cur.execute("""
                    INSERT INTO "players with 1k runs and 50 wickets" (player_name, format, total_runs, total_wickets)
                    VALUES (%s, %s, %s, %s)
                    """, (
                        player_name,
                        format,
                        total_runs[format],
                        total_wickets[format]
                    )
            )
conn.commit()
cur.close()
conn.close()


Connected to the database successfully
{14701: 'Nitish Kumar Reddy', 9647: 'Hardik Pandya', 11195: 'Shivam Dube', 12086: 'Abhishek Sharma', 587: 'Ravindra Jadeja', 8683: 'Shardul Thakur', 10945: 'Washington Sundar', 8808: 'Axar Patel', 12225: 'Cameron Green', 52495: 'Sam Konstas', 8646: 'Beau Webster', 13170: 'Aaron Hardie', 7662: 'Glenn Maxwell', 6250: 'Mitchell Marsh', 6557: 'Ben Stokes', 19636: 'Jacob Bethell', 12258: 'Will Jacks', 10045: 'Liam Livingstone', 6670: 'Chris Woakes', 8512: 'Jamie Overton', 13681: 'Gus Atkinson', 19767: 'Rehan Ahmed', 10420: 'Sam Curran'}
getting runs for player: {'headers': ['ROWHEADER', 'Test', 'ODI', 'T20', 'IPL'], 'values': [{'values': ['Matches', '10', '4', '4', '28']}, {'values': ['Innings', '16', '4', '3', '22']}, {'values': ['Runs', '396', '100', '90', '485']}, {'values': ['Balls', '691', '99', '50', '365']}, {'values': ['Highest', '114', '53', '74', '76']}, {'values': ['Average', '26.4', '33.33', '45', '28.53']}, {'values': ['SR', '57.31', '101.

# Question 12

In [ ]:
headers = {
	"x-rapidapi-key": "1d21170159mshe1410cd117264c9p1243d2jsn40d21f3b067a",
	"x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
}

conn = psycopg2.connect(**DB_CONFIG)
cur = conn.cursor()
print("Connected to the database successfully")

cur.execute("""
CREATE TABLE IF NOT EXISTS "international team performance" (
    team_name  VARCHAR(100),
    away_win VARCHAR(100),
    home_win VARCHAR(100)
    )
""")
international_team_url = f"https://cricbuzz-cricket.p.rapidapi.com/teams/v1/international"

response = requests.get(international_team_url, headers=headers)

int_team_list = response.json()

teams = int_team_list['list']

int_team_ids_with_teamName = {
    team['teamId']: [team['teamSName'], team['teamName']]
    for team in teams
    if 'teamId' in team and 'teamSName' in team and 'teamName' in team
}

print(int_team_ids_with_teamName)

for team_id, team_name in list(int_team_ids_with_teamName.items())[:5]:
    home_wins = 0
    away_wins = 0
    get_matches_url = f"https://cricbuzz-cricket.p.rapidapi.com/teams/v1/{team_id}/results"
    matches_response = requests.get(get_matches_url,headers=headers)
    matches_response_json = matches_response.json()
    for matches in matches_response_json.get('teamMatchesData', []):
        matchDetailsMap = matches.get('matchDetailsMap', {})
        for match in matchDetailsMap.get('match', []):
            match_info = match.get('matchInfo', {})
            match_Id = match_info.get('matchId',{})
            get_match_url = f"https://cricbuzz-cricket.p.rapidapi.com/mcenter/v1/{match_Id}"
            get_individual_match_response = requests.get(get_match_url, headers=headers)
            if get_individual_match_response.status_code != 200:
                continue
            match_data = get_individual_match_response.json()
            match_data = get_individual_match_response.json()
            individual_venue_info = match_data.get('venueinfo', {})
            individual_match_result = match_data.get('shortstatus')
            if individual_match_result is None:
                continue
            print(f"Team: {team_name}, Short Status: {individual_match_result}, Venue Country: {individual_venue_info.get('country')}")
            if individual_match_result and team_name[0] in individual_match_result:
                if team_name[1] in individual_venue_info.get('country'):
                    home_wins += 1
                else:
                    away_wins += 1
    cur.execute("""
        INSERT INTO "international team performance" (team_name, away_win, home_win)
        VALUES (%s, %s, %s)
    """, (team_name[1], away_wins, home_wins))
conn.commit()
cur.close()
conn.close()
    


Connected to the database successfully
{2: ['IND', 'India'], 96: ['AFG', 'Afghanistan'], 27: ['IRE', 'Ireland'], 3: ['PAK', 'Pakistan'], 4: ['AUS', 'Australia'], 5: ['SL', 'Sri Lanka'], 6: ['BAN', 'Bangladesh'], 9: ['ENG', 'England'], 10: ['WI', 'West Indies'], 11: ['RSA', 'South Africa'], 12: ['ZIM', 'Zimbabwe'], 13: ['NZ', 'New Zealand'], 71: ['MLY', 'Malaysia'], 72: ['NEP', 'Nepal'], 77: ['GER', 'Germany'], 161: ['NAM', 'Namibia'], 185: ['DEN', 'Denmark'], 190: ['SIN', 'Singapore'], 287: ['PNG', 'Papua New Guinea'], 298: ['KUW', 'Kuwait'], 300: ['VAN', 'Vanuatu'], 303: ['JER', 'Jersey'], 304: ['OMAN', 'Oman'], 343: ['FIJI', 'Fiji'], 527: ['ITA', 'Italy'], 529: ['BW', 'Botswana'], 541: ['BEL', 'Belgium'], 44: ['UGA', 'Uganda'], 26: ['CAN', 'Canada'], 7: ['UAE', 'United Arab Emirates'], 8: ['HKC', 'Hong Kong, China'], 14: ['KEN', 'Kenya'], 15: ['USA', 'United States of America'], 23: ['SCO', 'Scotland'], 24: ['NED', 'Netherlands'], 25: ['BER', 'Bermuda'], 675: ['IRN', 'Iran']}
Team: [

# Question 13

In [ ]:
headers = {
	"x-rapidapi-key": "673075d6ebmshf9d93a82582ea4fp1bfa13jsndc6cd297a25f",
	"x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
}
venue_data = {}
conn = psycopg2.connect(**DB_CONFIG)
cur = conn.cursor()
print("Connected to the database successfully")

# Create table
cur.execute("""
CREATE TABLE IF NOT EXISTS "partnership with 100+ runs" (
    batsman_one_name VARCHAR(100),
    batsman_two_name VARCHAR(100),
    runs INT,
    innings INT,
    Batting_team_name VARCHAR(100),
    match_id VARCHAR(100)  
)
""")

url = f"https://cricbuzz-cricket.p.rapidapi.com/teams/v1/international"
response = requests.get(url, headers=headers, timeout=5)
print(response.json())

for team in response.json()['list'][:2]:
    team_name = team['teamName']
    team_id = team.get('teamId', None)
    if team_id is not None:
        get_matchInfo_url = f"https://cricbuzz-cricket.p.rapidapi.com/teams/v1/{team_id}/results"
        match_response = requests.get(get_matchInfo_url, headers=headers, timeout=5)
        for matches in match_response.json().get('teamMatchesData', []):
            matchDetailsMap = matches.get('matchDetailsMap', {})
            for match in matchDetailsMap.get('match', []):
                match_info = match.get('matchInfo', {})
                match_id = match_info.get('matchId', {})
                get_score_card = f"https://cricbuzz-cricket.p.rapidapi.com/mcenter/v1/{match_id}/scard"
                scorecard_response = requests.get(get_score_card, headers=headers, timeout=5)
                if scorecard_response.status_code != 200:
                    continue
                scorecard_response_json = scorecard_response.json()
                scorecard_data = scorecard_response_json.get('scorecard', {})
                for inning in scorecard_data:
                    innings_Id = inning.get('inningsid', 0)
                    batsmen_team_name = inning.get('batteamname', '')
                    partnerships_data = inning.get('partnership', {})
                    partnerships_data = partnerships_data.get('partnership', [])
                    for partnership in partnerships_data:
                        runs = partnership.get('totalruns', 0)
                        batsman_one_name = partnership.get('bat1name', '')
                        batsman_two_name = partnership.get('bat2name', '')
                        if runs is not None and runs >= 100:
                            cur.execute("""
                                INSERT INTO "partnership with 100+ runs" (batsman_one_name, batsman_two_name, runs, innings, Batting_team_name, match_id)
                                VALUES (%s, %s, %s, %s, %s, %s)
                            """, (batsman_one_name, batsman_two_name, runs, innings_Id, batsmen_team_name, match_id))
conn.commit()
cur.close()
conn.close()

Connected to the database successfully
{'list': [{'teamName': 'Test Teams'}, {'teamId': 2, 'teamName': 'India', 'teamSName': 'IND', 'imageId': 776162, 'countryName': 'India'}, {'teamId': 96, 'teamName': 'Afghanistan', 'teamSName': 'AFG', 'imageId': 776177}, {'teamId': 27, 'teamName': 'Ireland', 'teamSName': 'IRE', 'imageId': 839366}, {'teamId': 3, 'teamName': 'Pakistan', 'teamSName': 'PAK', 'imageId': 776308, 'countryName': 'Pakistan'}, {'teamId': 4, 'teamName': 'Australia', 'teamSName': 'AUS', 'imageId': 776202, 'countryName': 'Australia'}, {'teamId': 5, 'teamName': 'Sri Lanka', 'teamSName': 'SL', 'imageId': 776254, 'countryName': 'Sri Lanka'}, {'teamId': 6, 'teamName': 'Bangladesh', 'teamSName': 'BAN', 'imageId': 776210}, {'teamId': 9, 'teamName': 'England', 'teamSName': 'ENG', 'imageId': 776237}, {'teamId': 10, 'teamName': 'West Indies', 'teamSName': 'WI', 'imageId': 776191}, {'teamId': 11, 'teamName': 'South Africa', 'teamSName': 'RSA', 'imageId': 776287}, {'teamId': 12, 'teamName'

# Question 14

In [42]:
headers = {
	"x-rapidapi-key": "6ff5242508mshf473990ba31d5d9p1081cejsn02b4ad82dbfd",
	"x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
}
conn = psycopg2.connect(**DB_CONFIG)
cur = conn.cursor()
print("Connected to the database successfully")

# Create table
cur.execute("""
CREATE TABLE IF NOT EXISTS "bowlers in same venue" (
    bowler_name VARCHAR(100),
    total_runs INT,
    total_overs INT,
    venue_name VARCHAR(100),
    economy_rate NUMERIC(5,2),
    total_matches_played INT
)
""")


venue_list = [11, 15, 19, 22, 24, 26, 27, 29, 31, 32, 33, 34, 35, 36, 40, 41, 42, 43, 46]
bowler_stats = {}

for venue_id in venue_list:
    get_venue_matches_url = f"https://cricbuzz-cricket.p.rapidapi.com/venues/v1/{venue_id}/matches"
    matches_response = requests.get(get_venue_matches_url, headers=headers, timeout=5)
    print(matches_response)
    if matches_response.status_code != 200:
        continue
    matches_data = matches_response.json()
    match_details = matches_data.get('matchDetails', [])
    print(match_details)
    for matches in match_details:
        match_details_map = matches.get('matchDetailsMap', {})
        print(match_details_map)
        for match in match_details_map.get('match', []):
            match_info = match.get('matchInfo', {})
            match_id = match_info.get('matchId', 0)
            print(f"Processing Match ID: {match_id}")
            venue_info = match_info.get('venueInfo', {})
            venue_name = venue_info.get('ground', '')
            get_score_card = f"https://cricbuzz-cricket.p.rapidapi.com/mcenter/v1/{match_id}/scard"
            scorecard_response = requests.get(get_score_card, headers=headers, timeout=5)
            if scorecard_response.status_code != 200:
                continue
            scorecard_response_json = scorecard_response.json()
            print(scorecard_response_json)
            scorecard_data = scorecard_response_json.get('scorecard', {})
            for inning in scorecard_data:
                bowling_data = inning.get('bowler',[])
                for bowler in bowling_data:
                    total_overs = float(bowler.get('overs', 0))
                    if total_overs < 4:
                        continue
                    bowler_name = bowler.get('name','')
                    total_wickets = bowler.get('wickets', 0)
                    economy = bowler.get('economy', 0)
                    runs = bowler.get('runs', 0)
                    bowler_stats[bowler_name] = bowler_stats.get(bowler_name, {
                        'total_runs': 0,
                        'total_overs': 0,
                        'total_wickets': 0,
                        'economy': 0,
                        'total_matches_played': 0
                    })
                    bowler_stats[bowler_name]['total_runs'] += int(runs)
                    bowler_stats[bowler_name]['total_overs'] += float(total_overs)
                    bowler_stats[bowler_name]['total_wickets'] += int(total_wickets)
                    bowler_stats[bowler_name]['economy'] += float(economy)
                    bowler_stats[bowler_name]['total_matches_played'] += 1

print(bowler_stats)

for bowler_name, stats in bowler_stats.items():
    if stats['total_matches_played'] >= 1:
        economy_rate = stats['economy'] / stats['total_matches_played']
        cur.execute("""
            INSERT INTO "bowlers in same venue" (bowler_name, total_runs, total_overs, venue_name, economy_rate, total_matches_played)
            VALUES (%s, %s, %s, %s, %s, %s)
        """, (
            bowler_name,
            stats['total_runs'],
            stats['total_overs'],
            venue_name,
            economy_rate,
            stats['total_matches_played']
        ))
conn.commit()

Connected to the database successfully
<Response [200]>
[{'matchDetailsMap': {'key': "ICC Men's T20 World Cup 2026", 'match': [{'matchInfo': {'matchId': 139252, 'seriesId': 11253, 'seriesName': "ICC Men's T20 World Cup 2026", 'matchDesc': '31st Match, Group D', 'matchFormat': 'T20', 'startDate': '1771306200000', 'endDate': '1771318800000', 'state': 'Complete', 'status': 'New Zealand won by 8 wkts', 'team1': {'teamId': 26, 'teamName': 'Canada', 'teamSName': 'CAN', 'imageId': 776227}, 'team2': {'teamId': 13, 'teamName': 'New Zealand', 'teamSName': 'NZ', 'imageId': 776333}, 'venueInfo': {'id': 11, 'ground': 'MA Chidambaram Stadium', 'city': 'Chennai', 'timezone': '+05:30'}, 'currBatTeamId': 13, 'seriesStartDt': '1770422400000', 'seriesEndDt': '1773100800000', 'isTimeAnnounced': True}, 'matchScore': {'team1Score': {'inngs1': {'inningsId': 1, 'runs': 173, 'wickets': 4, 'overs': 20.0}}, 'team2Score': {'inngs1': {'inningsId': 2, 'runs': 176, 'wickets': 2, 'overs': 15.0}}}}, {'matchInfo': {'ma

# Question 15